In [3]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0


In [4]:
pip install cupy-cuda12x

  Using cached fastrlock-0.8.3-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 MB 9.0 MB/s eta 0:00:0000:0100:01
Using cached fastrlock-0.8.3-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl (53 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cudf-cu12

  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 7.6 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 9.7 MB/s eta 0:00:00:00:0100:01
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━

In [2]:
import cupy as cp  # GPU-accelerated NumPy
import numpy as np
import os
import mne
import pandas as pd
import cudf  # GPU DataFrame processing (like Pandas)
from scipy.stats import skew, kurtosis
from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
from cupyx.scipy.fft import fft, ifft  # GPU FFT
import pywt  # Wavelet transform (still CPU-based)
from numba import cuda  # CUDA acceleration

In [3]:
# CUDA-accelerated Morlet wavelet transform
def morlet_wavelet(t, f, sigma=1.0):
    return cp.exp(2j * cp.pi * f * t) * cp.exp(-t**2 / (2 * sigma**2))

def manual_morlet_transform(data, scales, fs=1.0):
    t = cp.arange(len(data)) / fs
    transformed = []
    for scale in scales:
        wavelet = morlet_wavelet(t - cp.mean(t), scale)
        wavelet /= cp.linalg.norm(wavelet)  # Normalize wavelet
        convolved = ifft(fft(data) * fft(wavelet)).real
        transformed.append(convolved)
    return transformed

In [ ]:
def extract_channel_features(raw, condition, fmin=0.5, fmax=50):
    raw.pick('eeg')
    data = cp.asarray(raw.get_data())  # Move data to GPU
    channel_names = raw.ch_names
    features = {}
    scales = cp.linspace(2, 30, 10)
    fs = raw.info['sfreq']

    for i, ch in enumerate(channel_names):
        key_prefix = f'{condition}_{ch.lower()}'
        features[f'{key_prefix}_mean'] = cp.mean(data[i]).get()
        features[f'{key_prefix}_variance'] = cp.var(data[i]).get()
        features[f'{key_prefix}_skewness'] = skew(data[i].get())  # Skewness needs CPU
        features[f'{key_prefix}_kurtosis'] = kurtosis(data[i].get())  # Kurtosis needs CPU
        features[f'{key_prefix}_peak_to_peak'] = cp.ptp(data[i]).get()

        # FFT on GPU
        fft_values = cp.abs(fft(data[i]))
        features[f'{key_prefix}_fft_mean'] = cp.mean(fft_values).get()
        features[f'{key_prefix}_fft_std'] = cp.std(fft_values).get()
        features[f'{key_prefix}_fft_max'] = cp.max(fft_values).get()

        # GPU-based Wavelet Transform
        coeffs = manual_morlet_transform(data[i], scales, fs)
        wavelet_energy = sum(cp.sum(cp.square(c)).get() for c in coeffs)
        features[f'{key_prefix}_wavelet_energy'] = wavelet_energy

        wavelet_entropy = 0
        for c in coeffs:
            c = c.get()  # Move to CPU for entropy calculation
            c_norm = c / (np.sum(np.abs(c)) + 1e-10)
            c_norm = c[c > 0]  # Avoid log errors
            if len(c_norm) > 0:
                wavelet_entropy += -np.sum(c_norm * np.log2(c_norm))
        features[f'{key_prefix}_wavelet_entropy'] = wavelet_entropy

    # Compute PSD using Welch's method
    psd = raw.compute_psd(method='welch', fmin=fmin, fmax=fmax, n_fft=2048)
    psd_data = psd.get_data()
    freqs = psd.freqs
    psd_df = cudf.DataFrame(psd_data, columns=freqs, index=channel_names)  # Use cuDF for speed

    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'slow_alpha': (6, 9), 'alpha': (8, 12),
             'beta': (12, 30), 'gamma': (30, 50)}

    for band, (low, high) in bands.items():
        band_power = psd_df.loc[:, (freqs >= low) & (freqs <= high)].mean(axis=1)
        for ch in channel_names:
            key_prefix = f'{condition}_{ch.lower()}'
            features[f'{key_prefix}_{band}_power'] = band_power[ch]

    # Frontal Alpha Asymmetry (F3-F4)
    if 'F3' in channel_names and 'F4' in channel_names:
        features[f'{condition}f3_f4_alpha_asymmetry'] = features[f'{condition}_f4_alpha_power'] - features[f'{condition}_f3_alpha_power']

    return cudf.DataFrame([features])  # Return cuDF DataFrame

def process_and_combine(eo_file_path, ec_file_path, output_file):
    all_features = []

    raw_eo = mne.io.read_raw_fif(eo_file_path, preload=True)
    features_eo = extract_channel_features(raw_eo, "ec")
    all_features.append(features_eo)

    raw_ec = mne.io.read_raw_fif(ec_file_path, preload=True)
    features_ec = extract_channel_features(raw_ec, "eo")
    all_features.append(features_ec)

    combined_features = cudf.concat(all_features, axis=1)
    combined_features.to_pandas().to_csv(output_file, index=False)  # Convert to Pandas before saving
    print(f"Features saved to {output_file}")



Opening raw data file /home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 29)  idle
    Range : 0 ... 29999 =      0.000 ...    59.998 secs
Ready.
Reading 0 ... 29999  =      0.000 ...    59.998 secs...


/tmp/ipykernel_2237662/2993782915.py:61: RuntimeWarning: This filename (/home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eo = mne.io.read_raw_fif(eo_file_path, preload=True)


Effective window size : 4.096 (s)


ArrowInvalid: Can only convert 1-dimensional array values

In [5]:

def process_folder(source_folder, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    files = os.listdir(source_folder)
    
    ec_files = sorted([f for f in files if "restEC_eeg_1.fif" in f])
    eo_files = sorted([f for f in files if "restEO_eeg_1.fif" in f])

    for ec_file in ec_files:
        base_name = ec_file.replace("restEC_eeg_1.fif", "")
        eo_file = base_name + "restEO_eeg_1.fif"

        if eo_file in eo_files:
            ec_path = os.path.join(source_folder, ec_file)
            eo_path = os.path.join(source_folder, eo_file)
            output_path = os.path.join(destination_folder, base_name + "restcombined_eeg_1.csv")

            process_and_combine(ec_path, eo_path, output_path)
        else:
            print(f"Warning: No matching EO file found for {ec_file}")

In [5]:
pip install numpy==2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 10.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Run processing
source_folder = "/home/admincit/Desktop/Team_4/split_fif/mdd"
destination_folder = "/home/admincit/Desktop/Team_4/split_fif/mdd_processed"
process_folder(source_folder, destination_folder)


Opening raw data file /home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 29)  idle
    Range : 0 ... 29999 =      0.000 ...    59.998 secs
Ready.
Reading 0 ... 29999  =      0.000 ...    59.998 secs...


/tmp/ipykernel_2237662/2993782915.py:61: RuntimeWarning: This filename (/home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eo = mne.io.read_raw_fif(eo_file_path, preload=True)


Effective window size : 4.096 (s)


ArrowInvalid: Can only convert 1-dimensional array values